# Privacy Mechanisms

In this part, we discuss a few common mechanisms to achieve privacy, including de-identification, $k$-anonymity, and differential privacy.

## Privacy via De-Identification

Privacy preservation is not merely about the suppression of sensitive, personal identifiable information (PII); it is about how to release information in a way that is resistent to adversarial attacks / probes. Therefore, privacy perserving techniques need to make the fundamental tradeoff between data privacy and data utility -- that is, how to release data that is safe (to some degree) while also being useful for some forms of analyses.

For discussions in this chapter, we will use the [Adult dataset](https://archive.ics.uci.edu/dataset/20/census+income) with personal identifiable information (PII).

```{admonition} Data: Adult Dataset with Personal Identifiable Information
:class: note
- Location: "data/adult_with_pii.csv"
- Shape: (32563, 18)
- Note: this dataset contains additional PII beyond the original census data on UCI.
```

In [13]:
import pandas as pd

data = pd.read_csv('../data/adult_with_pii.csv')
data.head()

,Name,DOB,SSN,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,56,2174,0
1,Brandise Tripony,6/7/1988,150-19-2766,61523,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,0,0
2,Brenn McNeely,8/6/1991,725-59-9860,95668,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,32,0,0
3,Dorry Poter,4/6/2009,659-57-4974,25503,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,14,0,0
4,Dick Honnan,9/16/1951,220-93-3811,75387,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,72,0,0


As a first (and intuitive) step of privacy protection, it makes sense to remove some personal information that can be used to easily identify individuals. Here, ```SSN``` is a unique identifier of individuals and releasing it is a clear violation of individual privacy. Let's also remove ```Name``` and ```DOB``` in this process -- even though they are not unique identifiers of individuals, they are still quite specific and intuitively feel "personal". Removing these sensitive columns is a form of **de-identification**.

In [26]:
data_anonymized = data.drop(columns=['Name', 'DOB', 'SSN'])
data_anonymized.head()

,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
0,64152,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,56,2174,0
1,61523,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,0,0
2,95668,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,32,0,0
3,25503,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,14,0,0
4,75387,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,72,0,0


However, when thinking about privacy, it is important to realize that just personal identifiable information is typically not sufficient to protect individuals from being re-identified. **Re-identifiation** attack using a combination of features that are not immediately sensitive poses a significant threat in practice.

For instance, with a combination of ```Zip``` and ```Age```, you can typically uniquely identify an individual in this dataset. As a less obvious example, imagine you know someone with age 35 who lives in the United States, works in the government, and recently made some money from investment. Using just this information, you can narrow down to 19 individuals, among which _only one_ person had a positive capital gain. In other words, in this case, you have discovered _exactly_ how much money the person made from capital gain (2977) simply based on a combination of age, country of residence, broad professional category, and vague information on capital gain!

In [27]:
data_anonymized_attack = data_anonymized[
    (data_anonymized['Age'] == 35) & 
    (data_anonymized['Country'] == 'United-States') &
    (data_anonymized['Workclass'] == 'State-gov') &
    (data_anonymized['Capital Gain'] > 0)
]
data_anonymized_attack

,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
8390,18107,State-gov,Bachelors,13,Never-married,Exec-managerial,Not-in-family,Asian-Pac-Islander,Female,45,United-States,<=50K,35,2977,0


## $k$-Anonymity

In light of the above example, it is clear that (1) simply removing personal identifiable information may not be enough for privacy protection, and (2) we need to manage re-identification risk. One mechanism to do so is by **manipulating the data such that each individual "blends into" a group of non-distinguishable other individuals**. This is the key idea behind $k$-Anonymity {cite:p}`sweeney2002k`, defined as follows.

```{admonition} Definition: $k$-Anonymity
:class: tip
Let $D$ denote a dataset and $r \in D$ denote a particular row in the dataset. Given a value $k$ and a set of quasi-identifiers $QI$ (i.e., specific columns in the dataset that can be used to identify individuals), the dataset $D$ satisfies $k$-Anonymity if $\forall r \in D$, there are at least $k-1$ other rows in $D$ that share the same values with $r$ on quasi-identifiers $QI$.
```

Put differently, $k$-anonymity typically requires "obfuscating" the dataset such that when an attacker makes a query using some specific values of quasi-identifiers, the attacker will get back at least $k$ rows that are essentially indistinguishable from one another on those quasi-identifiers (so that any one of them cannot be uniquely identified).

It's easy to imagine that if we want to be "extremely conservative" and treat all columns of a dataset as quasi-identifiers, then enforcing $k$-anonymity may amount to manipulating data to such an extend that individuals become identical. This of course makes the data pretty much useless. In practice, therefore, implementing $k$-anonymity requires careful choices of (1) a reasonable value of $k$, (2) a reasonable set of quasi-identifiers, and (3) a reasonable approach of data manipulation.

How to achieve $k$-anonymity? One approach is **data generalization and suppression**, which is to modifying the values of quasi-identifiers to be _less specific_ (i.e., more "coarse"). For instance, modifying values of the ```Age``` variable from integers to ranges (e.g., changing 25 to "20-30") makes it much harder to uniquely identifying someone by age. However, algorithmically implementing $k$-anonymity is actually a very hard task -- in fact, it is NP-hard in general {cite:p}`meyerson2004complexity` and people often resort to _approximating_ $k$-anonymity. We omit the algorithmic details here, as they are beyond the scope of this book.

As a simple demonstration, we can apply this idea to coarsen the ```Age``` variable in the Adult dataset, and evaluate the same re-identification attack again.

In [28]:
# convert age to categorical with bucket size of 10
data_anonymized['Age'] = pd.cut(data_anonymized['Age'], bins=range(0, 101, 10))
data_anonymized.head()

,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
0,64152,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,"(50, 60]",2174,0
1,61523,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,"(30, 40]",0,0
2,95668,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,"(30, 40]",0,0
3,25503,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,"(10, 20]",0,0
4,75387,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,"(70, 80]",0,0


In [ ]:
data_anonymized_attack = data_anonymized[
    (data_anonymized['Age'] == pd.Interval(30,40)) & 
    (data_anonymized['Country'] == 'United-States') &
    (data_anonymized['Workclass'] == 'State-gov') &
    (data_anonymized['Capital Gain'] > 0)
    ]
# after coarsening, there are a number of individuals that match the criteria -- the attacker can no longer uniquely identify anyone
data_anonymized_attack

,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
4153,99817,State-gov,Some-college,10,Divorced,Prof-specialty,Unmarried,White,Female,50,United-States,<=50K,"(30, 40]",1506,0
7023,25621,State-gov,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,50,United-States,>50K,"(30, 40]",7688,0
8253,58074,State-gov,Masters,14,Divorced,Prof-specialty,Unmarried,Black,Female,40,United-States,>50K,"(30, 40]",7430,0
8390,18107,State-gov,Bachelors,13,Never-married,Exec-managerial,Not-in-family,Asian-Pac-Islander,Female,45,United-States,<=50K,"(30, 40]",2977,0
11820,82990,State-gov,Masters,14,Divorced,Prof-specialty,Unmarried,White,Male,30,United-States,<=50K,"(30, 40]",5455,0
12622,40322,State-gov,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,>50K,"(30, 40]",15024,0
15317,45807,State-gov,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,60,United-States,>50K,"(30, 40]",7688,0
18217,64300,State-gov,Assoc-voc,11,Married-civ-spouse,Prof-specialty,Husband,White,Male,40,United-States,>50K,"(30, 40]",4386,0
20285,37455,State-gov,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,80,United-States,>50K,"(30, 40]",99999,0
24319,21323,State-gov,Doctorate,16,Married-civ-spouse,Exec-managerial,Husband,White,Male,55,United-States,>50K,"(30, 40]",7688,0


## Differential Privacy

The previous two privacy mechanisms, namely de-identification and $k$-anonymity, seek to achieve privacy protection by _modifying the data_. In contrast, **differential privacy** represents a privacy mechanism of _algorithms_ acting on data. Roughly speaking, an algorithm satisfies differential privacy if it cannot be used to differentiate two datasets that differ by one data record (e.g., it cannot be used to identify whether one particular individual is present or not in a dataset). An important strength of differential privacy is its mathematical rigor. The above rough definition can be made very precise, which offers a clear approach to decide whether, and how much, an algorithm satisfies differential privacy. We will discuss the precise definition of differential privacy below.

To illustrate the key idea first, let's think about a very simple "algorithm", the averaging function $f(X) = \frac{1}{N} \sum_{x \in X} x$ that simply returns the average value of a particular variable $X$. For concreteness, we can think about each $x \in X$ being an indicator of whether an individual has cancer (which is highly sensitive information). In this case, $f(X)$ computes the proportion of cancer patients in a sample of $N$ individuals. Now, consider a particular individual with cancer indicator $x^*$ and two different datasets $D$ and $D'$ where $x^* \in D$ and $D' = D \setminus x^*$. In other words, the two datasets differ only by a single record $x^*$. By applying the averaging function on $D$ and $D'$, it is trivial to precisely recover the true value of $x^*$.[^footnote1] As a result, an attacker with the ability to query a dataset for average values with and without a particular row can precisely deduce the value of a sensitive feature.

```{admonition} Note about this example
:class: note
In discussions of differential privacy, it is common to assume that the attacker possesses basic / descriptive information of the dataset, such as its size, variable names / types, or even the values of certain variables. This is intentional because it corresponds to an "adversarial" context. The goal is to protect privacy even under such challenging contexts.
```

How do we protect privacy in the above example with an averaging function? Instead of modifying the original patient dataset, the proposal of differential privacy is to **add noise to the output of the algorithm**. Specifically, consider a "noisy" version of the averaging function, $F(X)$ (called a "mechanism"), defined simply as $F(X) = f(X) + \eta$. Note that $\eta$ here is not a fixed value but a random variable. Now the attacker querying the dataset with $F(X)$ will have much less ability to deduce the sensitive information of any individual, because the results from applying $F(X)$ are randomized each time, thereby effectively masking the true value of any specific $x$.

More precisely, differential privacy is often defined as the following property of a randomized function $F(.)$

```{admonition} Definition: Differential Privacy
:class: tip
A randomized function $F(.)$ satisfies $\varepsilon$-differential privacy if for all pairs of adjacent datasets $D$ and $D'$ and $S \subseteq Range(K)$

$$
\frac{\Pr(F(D) \in S)}{\Pr(F(D') \in S)} \leq e^{\varepsilon}
$$
```

Here, "adjacent datasets" means two datasets that differ only by one record (like in the example above), $Range(K)$ denote the range / possible values that $F(.)$ can return, and $\varepsilon \geq 0$ is a non-negative constant that controls "how much" privacy protection is guaranteed -- higher $\varepsilon$ indicates weaker privacy guarantee (e.g., imagine taking $\varepsilon$ to infinity, it implies that the two probabilities on the left-hand-side can be arbitrarily different).[^footnote2] 

Achieving the abovementioned $\varepsilon$-differential diversity hinges on picking the right distribution from which the noise $\eta$ is drawn. A commonly used distribution is the [**Laplace distribution**](https://en.wikipedia.org/wiki/Laplace_distribution), and the resulting differential privacy mechanism is known as the **Laplace mechanism**. In general, designing the proper mechanism to achieve differential privacy is a topic of ongoing research interest.

```{admonition} Maths behind the Laplace Mechanism (optional, toggle to show)
:class: dropdown
Why does the Laplace mechanism implements $\varepsilon$-differential privacy? Let $F(D)$ denote the randomized function querying dataset $D$. The Laplace mechanism computes it as $F(D) = f(D) + \eta$ where $\eta$ is drawn from a Laplace distribution with mean 0 and scale $\frac{s}{\varepsilon}$, that is, $\eta \sim Laplace \left(\frac{s}{\varepsilon} \right)$. Here, $\varepsilon$ represents the degree-of-privacy control (discussed above) and $s$ denotes the "sensitivity" of the actual query function $f(D)$, defined as $s := \max_{D,D'} \vert \vert f(D) - f(D') \vert \vert$.

For conciseness, let $b = \frac{s}{\varepsilon}$, then the density function of Laplace distribution can be written as $Laplace(x) = \frac{1}{2b} \exp \left( - \frac{|x|}{b} \right)$. For an arbitrary possible output value of the $F(.)$ that we denote as $y$, we will next show that $\frac{\Pr(F(D) = y)}{\Pr(F(D') = y)} \leq e^{\varepsilon}$.

Notice that[^footnote3] 
\begin{align*}
\Pr(F(D) = y) &=  \frac{1}{2b} \exp \left( - \frac{|y - f(D)|}{b} \right)\\
\Pr(F(D') = y) &=  \frac{1}{2b} \exp \left( - \frac{|y - f(D')|}{b} \right)
\end{align*}
Therefore, 

$$
\frac{\Pr(F(D) = y)}{\Pr(F(D') = y)} = \exp \left(\frac{|y-f(D')| - |y-f(D)|}{b} \right)
$$

By the [triangle inequality](https://en.wikipedia.org/wiki/Triangle_inequality), we have

$$
|y-f(D')| - |y-f(D)| \leq |(y-f(D')) - (y-f(D))| = |f(D) - f(D')|
$$

Subsequently, we have

$$
\frac{\Pr(F(D) = y)}{\Pr(F(D') = y)} \leq \exp \left(\frac{|f(D) - f(D')|}{b} \right) \leq \exp \left(\frac{s}{b} \right) = e^{\varepsilon}
$$

which completes the proof.
```

In practice, how much privacy is enough (that is, how low should we set $\varepsilon$)? This is a non-trivial question. While it might appear at first that more privacy protection is always better (suggesting setting $\varepsilon$ as low as possible), it is important to realize that doing so necessarily introduces greater noise into $F(.)$, making its returned values less useful / informative. In the extreme, we can achieve (almost) perfect privacy by completely overwhelming the true values of a query by noise. Here, we are directly faced with a first-order tradeoff between _privacy protection_ and _data utility_.

[^footnote1]: Let $f(X;D)$ and $f(X;D')$ be the average values of the two datasets, then $x^* = |D|f(X;D) - |D'|f(X;D')$.
[^footnote2]: A more general definition of differential privacy involves another parameter $\delta$. It states that a randomized function $F(.)$ satisfies $(\varepsilon, \delta)$-differential privacy if for all pairs of adjacent datasets $D$ and $D'$ and $S \subseteq Range(K)$ we have $\Pr(F(D) \in S) \leq e^{\varepsilon} \Pr(F(D') \in S) + \delta$. We discuss the simplified version with $\delta = 0$ here.
[^footnote3]: Technically the pdf function does not produce probability (but density / likelihood). A more rigorous proof would involve looking at an infinitesimal interval around $y$.